In [1]:
module EulerSystem2D

export ConservativeVariables, EigensystemX, EigenvaluesX,
   EigensystemY, EigenvaluesY, MaximumEigenvalue

# function Pressure(Q :: Transpose{Float64,Array{Float64,1}}, γ)
#    return (γ-1)*(Q[3] - Q[2].^2 ./ Q[1]/2)
# end
#
function Pressure(Q :: Array{Float64,1}, γ)
   return (γ-1)*(Q[4] - (Q[2].^2+Q[3].^2) ./ Q[1]/2)
end

function Pressure(Q :: Array{Float64,2}, γ)
   return (γ-1)*(Q[:,4] - (Q[:,2].^2+Q[:,3].^2) ./ Q[:,1]/2)
end

function Pressure(Q :: Array{Float64,3}, γ)
   return (γ-1)*(Q[:,:,4] - (Q[:,:,2].^2+Q[:,:,3].^2) ./ Q[:,:,1]/2)
end

function ConservativeVariables(Q, γ)
   return ConservativeVariables(Q[:,:,1], Q[:,:,2], Q[:,:,3], Q[:,:,4], γ)
end

function ConservativeVariables(ρ, u, v, p, γ)
   ρu = ρ.*u
   ρv = ρ.*v
   E = p/(γ-1) + ρ.*(u.^2+v.^2)/2
   return [ρ;;; ρu;;; ρv;;; E]
end

function PrimitiveVariables(Q :: Array{Float64,1}, γ)
   R = Q[1]
   U = Q[2]/R
   V = Q[3]/R
   P = Pressure(Q, γ)
   return R, U, V, P
end

function PrimitiveVariables(Q :: Array{Float64,2}, γ)
   R = Q[:,1]
   U = Q[:,2]./R
   V = Q[:,3]./R
   P = Pressure(Q, γ)
   return R, U, V, P
end

function PrimitiveVariables(Q :: Array{Float64,3}, γ)
   R = Q[:,:,1]
   U = Q[:,:,2]./R
   V = Q[:,:,3]./R
   P = Pressure(Q, γ)
   return R, U, V, P
end

function ArithmeticAverage(Q, γ)
   r = 3
   Qa = (Q[r,:] + Q[r+1,:])/2

   R, U, V, P = PrimitiveVariables(Qa, γ)
   A = sqrt.(γ*P/R)     # Sound speed
   H = (Qa[4] + P)/R    # Enthalpy
   h = 1/(2*H - U^2 - V^2)

   return U, V, A, H, h
end

function EigensystemX(Q, γ)
   Average = ArithmeticAverage
   #Average = RoeAverage

   U, V, A, H, h = Average(Q, γ)

   R = [1.0  U-A  V    H-U*A;
        1.0  U    V    (U^2+V^2)/2.0;
        0.0  0.0  1.0  V;
        1.0  U+A  V    H+U*A]

   I2A, Hh, Uh, Vh = 0.5/A, H*h, U*h, V*h

   L = [U*I2A+Hh-0.5  2.0-2.0*Hh  -V   Hh-0.5-U*I2A;
        -Uh-I2A       2.0*Uh      0.0  I2A-Uh;
        -Vh           2.0*Vh      1.0  -Vh;
        h             -2.0*h      0.0  h]

   return R, L
end

function EigenvaluesX(Q, γ)
   R, U, V, P = PrimitiveVariables(Q, γ)
   A = sqrt.(γ*P./R)

   return [U-A U U U+A]
end

function EigensystemY(Q, γ)
   Average = ArithmeticAverage
   #Average = RoeAverage

   U, V, A, H, h = Average(Q, γ)

   R = [1.0  U    V-A  H-V*A;
        1.0  U    V    (U^2+V^2)/2.0;
        0.0  1.0  0.0  U;
        1.0  U    V+A  H+V*A]

   I2A, Hh, Uh, Vh = 0.5/A, H*h, U*h, V*h

   L = [V*I2A+Hh-0.5  2.0-2.0*Hh  -U   Hh-0.5-V*I2A;
        -Uh           2.0*Uh      1.0  -Uh;
        -Vh-I2A       2.0*Vh      0.0  I2A-Vh;
        h             -2.0*h      0.0  h]

   return R, L
end

function EigenvaluesY(Q, γ)
   R, U, V, P = PrimitiveVariables(Q, γ)
   A = sqrt.(γ*P./R)

   return [V-A V V V+A]
end

function MaximumEigenvalue(Q, γ)
   R, U, V, P = PrimitiveVariables(Q, γ)
   A = sqrt.(γ*P./R)

   MU = max(maximum((U-A).^2), maximum((U+A).^2))
   MV = max(maximum((V-A).^2), maximum((V+A).^2))
   return sqrt(MU+MV)
end

end


Main.EulerSystem2D

In [2]:
using LinearAlgebra
using .EulerSystem2D

function Euler2D(Q, γ, Δx, Δy, CFL, FinalTime, Force, BoundaryConditionX,  BoundaryConditionY)
   Time = 0.0
   Δ = min(Δx, Δy)

   while Time < FinalTime
      Δt = CFL*Δ/MaximumEigenvalue(Q, γ)
      if Δt > (FinalTime - Time)
         Δt = FinalTime - Time
      end

      # 3rd order TVD Runge-Kutta scheme
      Q1 =      Q          -     Δt * (FluxDerivativeX(Q, γ, Δx, BoundaryConditionX)  + FluxDerivativeY(Q, γ, Δy, BoundaryConditionY)  .- Force(Q))
      Q2 = (3.0*Q +     Q1 -     Δt * (FluxDerivativeX(Q1, γ, Δx, BoundaryConditionX) + FluxDerivativeY(Q1, γ, Δy, BoundaryConditionY) .- Force(Q1))) / 4.0
      Q  = (    Q + 2.0*Q2 - 2.0*Δt * (FluxDerivativeX(Q2, γ, Δx, BoundaryConditionX) + FluxDerivativeY(Q2, γ, Δy, BoundaryConditionY) .- Force(Q2))) / 3.0

      Time  = Time + Δt;
   end
   return Q
end

function FluxDerivativeX(Q, γ, Δx, BoundaryConditionX)
   Ord = 5 # The order of the scheme
   N = size(Q, 1)
   M = size(Q, 2)
   F_half = zeros(N+1, M, 4)
   G_half = zeros(1, 4)
   #Qi = zeros(Ord+1, 3)   # Não adianta pré-alocar assim

   QB = BoundaryConditionX(Q)
   #M = MaximumEigenvalue(QB, γ)

   for j = 1:M
      for i = 1:N+1
         Qi = QB[i:i+Ord, j, :]
         Λ = EigenvaluesX(Qi, γ)
         M = maximum(abs.(Λ))

         R, L = EigensystemX(Qi, γ)

         W = Qi*L       # Transforms into characteristic variables
         G = Λ.*W       # The flux for the characteristic variables is Λ * L*QB
         print(W[:,1],"\n")
         for k = 1:4    # WENO reconstruction of the flux G
            G_half[k] = ReconstructedFlux(G[:,k], W[:,k], M)
         end

         F_half[i,j,:] = G_half*R # Brings back to conservative variables
      end
   end

   return (F_half[2:end,:,:] - F_half[1:end-1,:,:]) / Δx # Derivative of Flux
end

function FluxDerivativeY(Q, γ, Δy, BoundaryConditionY)
   Ord = 5 # The order of the scheme
   N = size(Q, 1)
   M = size(Q, 2)
   F_half = zeros(N, M+1, 4)
   G_half = zeros(1, 4)
   #Qj = zeros(Ord+1, 3)   # Não adianta pré-alocar assim

   QB = BoundaryConditionY(Q)
   #M = MaximumEigenvalue(QB, γ)

   for j = 1:M+1
      for i = 1:N
         Qj = QB[i, j:j+Ord, :]
         Λ = EigenvaluesY(Qj, γ)
         M = maximum(abs.(Λ))

         R, L = EigensystemY(Qj, γ)

         W = Qj*L       # Transforms into characteristic variables
         G = Λ.*W       # The flux for the characteristic variables is Λ * L*QB
         for k = 1:4    # WENO reconstruction of the flux G
            G_half[k] = ReconstructedFlux(G[:,k], W[:,k], M)
         end

         F_half[i,j,:] = G_half*R # Brings back to conservative variables
      end
   end

   return (F_half[:,2:end,:] - F_half[:,1:end-1,:]) / Δy # Derivative of Flux
end

function ReconstructedFlux(F, Q, M)
   ReconstructionLTR = WenoZ5ReconstructionLTR
   #ReconstructionLTR = WenoM5ReconstructionLTR

   F_plus  = (F + M*Q)/2
   F_minus = (F - M*Q)/2

   F_half_plus  = ReconstructionLTR(F_plus)
   F_half_minus = ReconstructionLTR(F_minus[end:-1:1])

   return F_half_plus + F_half_minus
end

function WenoZ5ReconstructionLTR(Q)
   ɛ = 10.0^(-40)
   # Calcula os indicadores de suavidade locais
   β0 = (1/2*Q[1] - 2*Q[2] + 3/2*Q[3])^2 + 13/12*(Q[1] - 2*Q[2] + Q[3])^2
   β1 = (-1/2*Q[2] + 1/2*Q[4])^2 + 13/12*(Q[2] - 2*Q[3] + Q[4])^2
   β2 = (-3/2*Q[3] + 2*Q[4] - 1/2*Q[5])^2 + 13/12*(Q[3] - 2*Q[4] + Q[5])^2
   # Calcula o indicador de suavidade global
   τ = abs(β0 - β2)
   # Calcula os pesos do WENO-Z
   α0 = (1/10) * (1 + (τ/(β0 + ɛ))^2)
   α1 = (6/10) * (1 + (τ/(β1 + ɛ))^2)
   α2 = (3/10) * (1 + (τ/(β2 + ɛ))^2)
   sum_α = α0 + α1 + α2
   ω0 = α0 / sum_α
   ω1 = α1 / sum_α
   ω2 = α2 / sum_α
   # Calcula os fhat em cada subestêncil
   fhat0 = (2*Q[1] - 7*Q[2] + 11*Q[3])/6
   fhat1 = (-Q[2] + 5*Q[3] + 2*Q[4])/6
   fhat2 = (2*Q[3] + 5*Q[4] - Q[5])/6
   #Finalmente, calcula o fhat do estêncil todo
   return ω0*fhat0 + ω1*fhat1 + ω2*fhat2
end

g(ω, d) = (ω*(d+d^2-3*d*ω+ω^2))/(d^2+ω*(1-2*d))

function WenoM5ReconstructionLTR(Q)
   ɛ = 10.0^(-40)
   # Calcula os indicadores de suavidade locais
   β0 = (1/2*Q[1] - 2*Q[2] + 3/2*Q[3])^2 + 13/12*(Q[1] - 2*Q[2] + Q[3])^2
   β1 = (-1/2*Q[2] + 1/2*Q[4])^2 + 13/12*(Q[2] - 2*Q[3] + Q[4])^2
   β2 = (-3/2*Q[3] + 2*Q[4] - 1/2*Q[5])^2 + 13/12*(Q[3] - 2*Q[4] + Q[5])^2
   # Calcula os pesos do WENO-JS
   α0 = (1/10) * (1/(β0 + ɛ)^2)
   α1 = (6/10) * (1/(β1 + ɛ)^2)
   α2 = (3/10) * (1/(β2 + ɛ)^2)
   sum_α = α0 + α1 + α2
   ω0 = α0 / sum_α
   ω1 = α1 / sum_α
   ω2 = α2 / sum_α
   # Mapeia
   α0 = g(ω0, 1/10)
   α1 = g(ω1, 6/10)
   α2 = g(ω2, 3/10)
   sum_α = α0 + α1 + α2
   ω0 = α0 / sum_α
   ω1 = α1 / sum_α
   ω2 = α2 / sum_α
   # Calcula os fhat em cada subestêncil
   fhat0 = (2*Q[1] - 7*Q[2] + 11*Q[3])/6
   fhat1 = (-Q[2] + 5*Q[3] + 2*Q[4])/6
   fhat2 = (2*Q[3] + 5*Q[4] - Q[5])/6
   #Finalmente, calcula o fhat do estêncil todo
   return ω0*fhat0 + ω1*fhat1 + ω2*fhat2
end

function PeriodicGhostPointsX(Q)
   Qg = [Q[end-2:end,:,:]; Q; Q[1:3,:,:]]
end

function PeriodicGhostPointsY(Q)
   Qg = [Q[:,end-2:end,:] Q Q[:,1:3,:]]
end


PeriodicGhostPointsY (generic function with 1 method)

In [3]:
using PlotlyJS

function MalhaRetangular(N)
   Δ = 1.0/N
   Δx = Δ; Δy = Δ
   x = 0.0:Δx:0.25
   y = Δy:Δy:1.0-Δy

   X = repeat(reshape(x, length(x), 1), 1, length(y))
   Y = repeat(reshape(y, 1, length(y)), length(x), 1)

   return X, Y, Δ
end

function CondiçãoInicialRayleighTaylor(X :: Array{Float64, 2}, Y :: Array{Float64, 2}, γ)
   R = zeros(size(X))
   P = zeros(size(X))
   U = zeros(size(X))
   V = zeros(size(X))
   for i = eachindex(R)
      if Y[i] < 0.5
         R[i] = 2.0
         P[i] = 2.0*Y[i] + 1.0
      else
         R[i] = 1.0
         P[i] = Y[i] + 1.5
      end
      a = √(γ * P[i] / R[i])
      V[i] = -0.025 * a * cos(8.0*π*X[i]);
   end

   E = P/(γ-1.0) + R.*(U.^2 + V.^2)/2.0

   #Q0 = cat(R, R.*U, R.*V, E, dims=3)
   Q0 = [R;;; R.*U;;; R.*V;;; E]
   return Q0
end

function CondiçãoInicialRayleighTaylor(N :: Int64, γ)
   X, Y, Δ = MalhaRetangular(N)
   Q0 = CondiçãoInicialRayleighTaylor(X, Y, γ)
   return X, Y, Δ, Q0
end

function RayleighTaylorGravity(Q :: Array{Float64,3})
   g = -1.0
   # for i = 1:size(U,1)
   #    for j = 1:size(U,2)
   #       F[i,j,3] = -g * U[i,j,1]
   #       F[i,j,4] = -g * U[i,j,3]
   #    end
   # end
   Z = zeros(size(Q,1), size(Q,2), 1)
   F = [Z;;;Z;;;-g*Q[:,:,1];;;-g*Q[:,:,3]]
   return F
end

function RayleighTaylorGhostPointsX(Q)
   #c = [1.0,-1.0,1.0,1.0]
   # for i = 1:3; for k = 1:4; Ug[i,k] = c[k]*U[end-i,j,k]; end; end
   # for i = 4:size(Ug,1)-3; for k = 1:4; Ug[i,k] = U[i-3,j,k]; end; end
   # for i = -2:0; for k = 1:4; Ug[size(Ug,1)+i,k] = c[k]*U[1-i,j,k]; end; end
   Qg = [[Q[3:-1:1, :, 1];;;-Q[3:-1:1, :, 2];;;Q[3:-1:1, :, 3];;;Q[3:-1:1, :, 4]];
         Q;
         [Q[end:-1:end-2, :, 1];;;-Q[end:-1:end-2, :, 2];;;Q[end:-1:end-2, :, 3];;;Q[end:-1:end-2, :, 4]]]
   return Qg
end

function RayleighTaylorGhostPointsY(Q, γ)
   # for j = 1:3
   #    Ug[j,1] = 2.0
   #    Ug[j,2] = 0.0
   #    Ug[j,3] = 0.0
   #    Ug[j,4] = 1.0/(5.0/3.0 - 1.0)
   # end
   # for j = 4:size(Ug,1)-3; for k = 1:4; Ug[j,k] = U[i,j-3,k]; end; end
   # for j = -2:0
   #    Ug[size(Ug,1)+j,1] = 1.0
   #    Ug[size(Ug,1)+j,2] = 0.0
   #    Ug[size(Ug,1)+j,3] = 0.0
   #    Ug[size(Ug,1)+j,4] = 2.5/(5.0/3.0 - 1.0)
   # end
   M = size(Q,1)
   Qg = [repeat([2.0;;;0.0;;;0.0;;;1.0/(γ-1.0)], M, 3);;
         Q;;
         repeat([1.0;;;0.0;;;0.0;;;2.5/(γ-1.0)], M, 3)]
   return Qg
end

function grafico(x, y, U, título)
   x_list=zeros(size(x,1)*size(x,2))
   y_list=zeros(size(y,1)*size(y,2))
   z_list=zeros(size(x,1)*size(x,2))

   for i in 1:size(x,1)
      for j in 1:size(x,2)
         x_list[(i-1)*size(x,1)+j]=x[i,j]
         y_list[(i-1)*size(x,1)+j]=y[i,j]
         z_list[(i-1)*size(x,1)+j]=U[i,j,1]
      end
   end
   
   plot(
      heatmap(
         x=x_list,
         y=y_list,
         z=z_list,
     ),
     st = :heatmap, aspect_ratio = 0.75, title = título, size = (800,800))
end

function RayleighTaylor(N :: Integer)
   γ = 5.0/3.0
   #N = 160
   x, y, Δ, U0 = CondiçãoInicialRayleighTaylor(N, γ)
   cfl = 0.6
   t_final = 0.001
   GhostPointsX(U) = RayleighTaylorGhostPointsX(U)
   GhostPointsY(U) = RayleighTaylorGhostPointsY(U, γ)
   U = Euler2D(U0, γ, Δ, Δ, cfl, t_final, RayleighTaylorGravity, GhostPointsX, GhostPointsY)
   grafico(x, y, U, "T = $t_final")
end


WebIO._IJuliaInit()

RayleighTaylor (generic function with 1 method)

In [4]:
γ = 5.0/3.0
N = 160
x, y, Δ, U0 = CondiçãoInicialRayleighTaylor(N, γ)
cfl = 0.6
t_final = 0.04
GhostPointsX(U) = RayleighTaylorGhostPointsX(U)
GhostPointsY(U) = RayleighTaylorGhostPointsY(U, γ)
U = Euler2D(U0, γ, Δ, Δ, cfl, t_final, RayleighTaylorGravity, GhostPointsX, GhostPointsY)
print("finished")

[0.6000004990551243, 0.6000000315785328, 0.5999999999999999, 0.5999999999999999, 0.6000000315785328, 0.6000004990551243]
[0.6000000000000002, 0.6000000000000002, 0.6000000000000002, 0.6000000000000002, 0.6000003735185568, 0.600002195351475]
[0.6000001255365481, 0.6000001255365481, 0.6, 0.6, 0.6000012095295074, 0.6000052871645086]
[0.6000011113580703, 0.6000007378396256, 0.6, 0.6, 0.6000024261979108, 0.6000093463082393]
[0.600003598804211, 0.6000017769723319, 0.6, 0.6, 0.6000039044273081, 0.6000139754439737]
[0.6000072188484392, 0.6000031412166391, 0.5999999999999999, 0.6, 0.6000054995178972, 0.6000187214385672]
[0.6000116171343869, 0.6000046970304447, 0.5999999999999999, 0.6, 0.6000070553309143, 0.6000231197206687]
[0.6000163631266343, 0.6000062921197414, 0.6000000000000001, 0.6000000000000001, 0.6000084195726532, 0.6000267397563278]
[0.6000209922547263, 0.6000077703462585, 0.6, 0.6, 0.6000094587019918, 0.600029227192449]
[0.6000250513884376, 0.6000089870112946, 0.6000000000000001, 0.6

Excessive output truncated after 524296 bytes.

0.30000036891981285, 0.30000055567903516]
[0.30000264358225437, 0.3000006047647537, 0.3, 0.3, 0.30000006276827407, 0.3]
[0.30000109767573735, 0.30000018675927825, 0.29999999999999993, 0.29999999999999993, 0.29999999999999993, 0.30000018675927825]
[0.30000018675927825, 0.29999999999999993, 0.29999999999999993, 0.29999999999999993, 0.30000018675927825, 0.30000109767573735]
[0.3, 0.30000006276827407, 0.3, 0.3, 0.3000006047647537, 0.30000264358225437]
[0.30000055567903516, 0.30000036891981285, 0.3, 0.3, 0.3000012130989554, 0.3000046731541197]
[0.30000179940210536, 0.3000008884861658, 0.2999999999999999, 0.2999999999999999, 0.3000019522136539, 0.3000069877219867]
[0.3000036094242196, 0.30000157060831956, 0.29999999999999993, 0.29999999999999993, 0.3000027497589486, 0.3000093607192835]
[0.3000058085671934, 0.30000234851522234, 0.29999999999999993, 0.29999999999999993, 0.30000352766545707, 0.3000115598603343]
[0.3000081815633172, 0.3000031460598707, 0.30000000000000004, 0.30000000000000004, 0

In [5]:
size(U0)

(41, 159, 4)

In [6]:
x_list=zeros(size(x,1)*size(x,2))
y_list=zeros(size(y,1)*size(y,2))
z_list=zeros(size(x,1)*size(x,2))

for i in 1:size(x,1)
   for j in 1:size(x,2)
      x_list[(i-1)*size(x,2)+j]=i
      y_list[(i-1)*size(x,2)+j]=j
      z_list[(i-1)*size(x,2)+j]=U[i,j,1]
   end
end

In [7]:
plot(
      heatmap(
         x=x_list,
         y=y_list,
         z=z_list,
     ))

data: [
  "heatmap with fields transpose, type, x, y, and z"
]

layout: "layout with fields margin and template"

In [8]:
RayleighTaylor(160)

[0.6000004990551243, 0.6000000315785328, 0.5999999999999999, 0.5999999999999999, 0.6000000315785328, 0.6000004990551243]
[0.6000000000000002, 0.6000000000000002, 0.6000000000000002, 0.6000000000000002, 0.6000003735185568, 0.600002195351475]
[0.6000001255365481, 0.6000001255365481, 0.6, 0.6, 0.6000012095295074, 0.6000052871645086]
[0.6000011113580703, 0.6000007378396256, 0.6, 0.6, 0.6000024261979108, 0.6000093463082393]
[0.600003598804211, 0.6000017769723319, 0.6, 0.6, 0.6000039044273081, 0.6000139754439737]
[0.6000072188484392, 0.6000031412166391, 0.5999999999999999, 0.6, 0.6000054995178972, 0.6000187214385672]
[0.6000116171343869, 0.6000046970304447, 0.5999999999999999, 0.6, 0.6000070553309143, 0.6000231197206687]
[0.6000163631266343, 0.6000062921197414, 0.6000000000000001, 0.6000000000000001, 0.6000084195726532, 0.6000267397563278]
[0.6000209922547263, 0.6000077703462585, 0.6, 0.6, 0.6000094587019918, 0.600029227192449]
[0.6000250513884376, 0.6000089870112946, 0.6000000000000001, 0.6

Excessive output truncated after 524296 bytes.

 0.30000036891981285, 0.30000055567903516]
[0.30000264358225437, 0.3000006047647537, 0.3, 0.3, 0.30000006276827407, 0.3]
[0.30000109767573735, 0.30000018675927825, 0.29999999999999993, 0.29999999999999993, 0.29999999999999993, 0.30000018675927825]
[0.30000018675927825, 0.29999999999999993, 0.29999999999999993, 0.29999999999999993, 0.30000018675927825, 0.30000109767573735]
[0.3, 0.30000006276827407, 0.3, 0.3, 0.3000006047647537, 0.30000264358225437]
[0.30000055567903516, 0.30000036891981285, 0.3, 0.3, 0.3000012130989554, 0.3000046731541197]
[0.30000179940210536, 0.3000008884861658, 0.2999999999999999, 0.2999999999999999, 0.3000019522136539, 0.3000069877219867]
[0.3000036094242196, 0.30000157060831956, 0.29999999999999993, 0.29999999999999993, 0.3000027497589486, 0.3000093607192835]
[0.3000058085671934, 0.30000234851522234, 0.29999999999999993, 0.29999999999999993, 0.30000352766545707, 0.3000115598603343]
[0.3000081815633172, 0.3000031460598707, 0.30000000000000004, 0.30000000000000004, 

LoadError: MethodError: no method matching Plot(::GenericTrace{Dict{Symbol, Any}}, ::Layout{Dict{Symbol, Any}}, ::Vector{PlotlyFrame}; st=:heatmap, aspect_ratio=0.75, title="T = 0.001", size=(800, 800))
[0mClosest candidates are:
[0m  Plot(::AbstractTrace, ::Any, ::AbstractVector{<:PlotlyFrame}; config) at C:\Users\silva\.julia\packages\PlotlyBase\xb3Du\src\PlotlyBase.jl:110[91m got unsupported keyword arguments "st", "aspect_ratio", "title", "size"[39m
[0m  Plot(::AbstractTrace, ::Any) at C:\Users\silva\.julia\packages\PlotlyBase\xb3Du\src\PlotlyBase.jl:110[91m got unsupported keyword arguments "st", "aspect_ratio", "title", "size"[39m
[0m  Plot([91m::TT[39m, ::TL, ::TF, [91m::Base.UUID[39m, [91m::PlotConfig[39m) where {TT<:(AbstractVector{<:AbstractTrace}), TL<:AbstractLayout, TF<:(AbstractVector{<:PlotlyFrame})} at C:\Users\silva\.julia\packages\PlotlyBase\xb3Du\src\PlotlyBase.jl:79[91m got unsupported keyword arguments "st", "aspect_ratio", "title", "size"[39m
[0m 